### MNIST数据集简介
官网：http://yann.lecun.com/exdb/mnist/ 包含4个文件：训练集图像、训练集标签、测试集图像、测试集标签   
简介：10个类别的手写数字图像数据集，每张图像是28×28的灰度图像784个像素点，训练集60000张图像，测试10000张  

In [8]:
# 1 导入需要的模块、包等
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from torch.optim import sgd, adam



In [2]:
# 参数设置
batch_size = 64
learning_rate = 0.001
num_epoch = 100


In [9]:
# 2 数据处理
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5
    x = x.reshape((-1,))
    x = torch.from_numpy(x)
    return x

In [10]:

train_set = mnist.MNIST('./data', train=True, transform=data_tf, download=True)
test_set = mnist.MNIST('./data', train=False, transform=data_tf, download=True)

# 使用 pytorch 自带的 DataLoader 定义一个数据迭代器
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=64, shuffle=False)


In [11]:
# 创建模型
class SingleHiddenNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, out_dim):
        super(SingleHiddenNN, self).__init__()
        self.hidden = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Sigmoid(),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, x):
        net_out = self.hidden(x)
        return net_out


In [12]:
# 定义损失函数
model = SingleHiddenNN(784, 20, 10)
criterion = nn.CrossEntropyLoss()
opt = sgd.SGD(model.parameters(), lr=learning_rate)

In [13]:
# 训练测试

train_loss_list = []
test_loss_list = []

train_acc_list = []
test_acc_list = []

for i in range(num_epoch):

    for im, label in train_data:
        # 前向传播
        out = model(im)
        loss = criterion(out, label)

        # 反向传播
        opt.zero_grad()
        loss.backward()
        opt.step()

        # 计算分类的准确率:
        """
        # 分析一下torch.max(input,dim),返回最大值和最大值的索引
        # 参数input是一个多分类的tensor; 参数dim是轴为0或1,0取每列的最大值，1取每行的最大值
        """

        # 计算行最大值索引
        _, pred = torch.max(out, 1)

        # 判断索引与标签一致的个数
        num_correct = (pred == label).sum().item()

        # 计算准确率： 预测正确的个数除以样本数
        acc = num_correct / im.shape[0]

        train_loss_list.append(loss)
        train_acc_list.append(acc)
    # print("迭代次数：%d,loss值：%.6f,准确率：%.6f" % (i, loss.item(), acc))

    model.eval()
    for im_test, label_test in test_data:
        # 前向传播
        out_test = model(im_test)
        loss_test = criterion(out_test, label_test)

        # 计算分类的准确率:
        # 计算行最大值索引
        _, pred_test = torch.max(out_test, 1)

        # 判断索引与标签一致的个数
        num_correct_test = (pred_test == label_test).sum().item()

        # 计算准确率： 预测正确的个数除以样本数
        acc_test = num_correct_test / im_test.shape[0]

        test_loss_list.append(loss_test)
        test_acc_list.append(acc_test)
    print("迭代次数：%d,train_loss值：%.6f,训练准确率：%.6f,test_loss值：%.6f,测试准确率：%.6f"
          % (i, loss.item(), acc, loss_test.item(), acc_test))


迭代次数：0,train_loss值：2.269714,训练准确率：0.250000,test_loss值：2.226971,测试准确率：0.250000
迭代次数：1,train_loss值：2.141471,训练准确率：0.531250,test_loss值：2.128467,测试准确率：0.500000
迭代次数：2,train_loss值：2.105304,训练准确率：0.468750,test_loss值：2.029823,测试准确率：0.687500
迭代次数：3,train_loss值：2.010177,训练准确率：0.625000,test_loss值：1.931293,测试准确率：0.750000
迭代次数：4,train_loss值：1.855277,训练准确率：0.718750,test_loss值：1.836591,测试准确率：0.750000
迭代次数：5,train_loss值：1.830461,训练准确率：0.593750,test_loss值：1.744003,测试准确率：0.750000
迭代次数：6,train_loss值：1.805002,训练准确率：0.562500,test_loss值：1.658004,测试准确率：0.750000
迭代次数：7,train_loss值：1.702817,训练准确率：0.625000,test_loss值：1.574552,测试准确率：0.750000
迭代次数：8,train_loss值：1.612138,训练准确率：0.781250,test_loss值：1.498572,测试准确率：0.812500
迭代次数：9,train_loss值：1.650608,训练准确率：0.625000,test_loss值：1.424141,测试准确率：0.812500
迭代次数：10,train_loss值：1.471683,训练准确率：0.718750,test_loss值：1.354732,测试准确率：0.812500
迭代次数：11,train_loss值：1.298552,训练准确率：0.781250,test_loss值：1.289734,测试准确率：0.812500
迭代次数：12,train_loss值：1.229094,训练准确率：0.843750,test_loss值：1.22950